In [15]:
import pandas as pd
import numpy as np

In [16]:
class PCAnalyzer():
    def __init__(self,n_components):
        self.n_components = n_components
        
    def fit(self, data):
        # data of dimension (n_samples, n_features)
        # Assuming n_samples > n_features (TODO: handle for the case of n_fatures <= n_samples)
        self.mean = np.mean(data, axis=0)
        self.covmat = np.cov(np.transpose(data))
        self.lamda, self.U = np.linalg.eigh(self.covmat)
        # Get eigen vectors and eigen values in eigen value descending order
        idx = self.lamda.argsort()[::-1]
        self.U = self.U[:,idx]
        self.lamda = self.lamda[idx]
        self.n_features = data.shape[1]
    
    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)  
    
    def inverse_transform(self, transformed_data):
        # Convert back to original coordinates
        return np.transpose(np.matmul(self.U,np.transpose(transformed_data))+np.expand_dims(self.mean,1))
    
    def transform(self, data):
        # P = U.t * (X.t-mean(X.t)) with only first n_components PC
        return np.transpose(np.matmul(np.transpose(self.U), (np.transpose(data)-np.expand_dims(self.mean,1)))
    *np.expand_dims(np.concatenate((np.ones(self.n_components),np.zeros(self.n_features-self.n_components))),1))
    
        


In [17]:
def array_name(obj, namespace):
    return [name for name in namespace if namespace[name] is obj][0]

In [18]:
# Read input data
dataI = np.array(pd.read_csv("data\dataI.csv")[['X1','X2','X3','X4']])
dataII = np.array(pd.read_csv("data\dataII.csv")[['X1','X2','X3','X4']])
dataIII = np.array(pd.read_csv("data\dataIII.csv")[['X1','X2','X3','X4']])
dataIV = np.array(pd.read_csv("data\dataIV.csv")[['X1','X2','X3','X4']])
dataV = np.array(pd.read_csv("data\dataV.csv")[['X1','X2','X3','X4']])
iris = np.array(pd.read_csv("data\iris.csv")[['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width']])

In [19]:
for n in range(4):
    pca_noiseless = PCAnalyzer(n+1)
    pca_noiseless.fit(iris)
    for data in (dataI, dataII, dataIII, dataIV, dataV):
        data_name = array_name(data, globals())
        mean_square_error = np.mean(np.sum((pca_noiseless.inverse_transform(pca_noiseless.transform(data)) - iris) ** 2,axis=1))
        print("Train with iris data set -> mean square error of " + data_name +" with "+str(n+1) +" PC: "+str(mean_square_error))

for n in range(4):
    pca_noisy = PCAnalyzer(n+1)
    for data in (dataI, dataII, dataIII, dataIV, dataV):
        data_name = array_name(data, globals())
        mean_square_error = np.mean(np.sum((pca_noisy.inverse_transform(pca_noisy.fit_transform(data)) - iris) ** 2,axis=1))
        print("Train with noisy data set -> mean square error of " + data_name +" with "+str(n+1) +" PC: "+str(mean_square_error))

Train with iris data set -> mean square error of dataI with 1 PC: 0.6410931849009849
Train with iris data set -> mean square error of dataII with 1 PC: 1.2903724507598
Train with iris data set -> mean square error of dataIII with 1 PC: 0.7999427437338247
Train with iris data set -> mean square error of dataIV with 1 PC: 1.917767749946061
Train with iris data set -> mean square error of dataV with 1 PC: 0.38345031150498454
Train with iris data set -> mean square error of dataI with 2 PC: 0.7156284875049558
Train with iris data set -> mean square error of dataII with 2 PC: 1.9672403923798694
Train with iris data set -> mean square error of dataIII with 2 PC: 0.8280825547067442
Train with iris data set -> mean square error of dataIV with 2 PC: 3.331722103940333
Train with iris data set -> mean square error of dataV with 2 PC: 0.1755630002443389
Train with iris data set -> mean square error of dataI with 3 PC: 0.9083929073982754
Train with iris data set -> mean square error of dataII with 

In [20]:
# Export 2PC for dataII
pca_data2 = PCAnalyzer(2)

X_hat = pca_data2.inverse_transform(pca_data2.fit_transform(data))
np.savetxt("xl55-recon.csv", np.transpose(X_hat), delimiter =',', header ='Sepal.Length,Sepal.Width,Petal.Length,Petal.Width',comments='')

In [21]:
# Compare with sklearn PCA

from sklearn.decomposition import PCA 

for n in range(4):
    pca_noiseless = PCA(n+1)
    pca_noiseless.fit(iris)
    for data in (dataI, dataII, dataIII, dataIV, dataV):
        data_name = array_name(data, globals())
        mean_square_error = np.mean(np.sum((pca_noiseless.inverse_transform(pca_noiseless.transform(data)) - iris) ** 2,axis=1))
        print("Train with iris data set -> mean square error of " + data_name +" with "+str(n+1) +" PC: "+str(mean_square_error))

for n in range(4):
    pca_noisy = PCA(n+1)
    for data in (dataI, dataII, dataIII, dataIV, dataV):
        data_name = array_name(data, globals())
        mean_square_error = np.mean(np.sum((pca_noisy.inverse_transform(pca_noisy.fit_transform(data)) - iris) ** 2,axis=1))
        print("Train with noisy data set -> mean square error of " + data_name +" with "+str(n+1) +" PC: "+str(mean_square_error))


Train with iris data set -> mean square error of dataI with 1 PC: 0.6410931849009851
Train with iris data set -> mean square error of dataII with 1 PC: 1.2903724507598011
Train with iris data set -> mean square error of dataIII with 1 PC: 0.7999427437338249
Train with iris data set -> mean square error of dataIV with 1 PC: 1.9177677499460624
Train with iris data set -> mean square error of dataV with 1 PC: 0.3834503115049846
Train with iris data set -> mean square error of dataI with 2 PC: 0.7156284875049566
Train with iris data set -> mean square error of dataII with 2 PC: 1.9672403923798711
Train with iris data set -> mean square error of dataIII with 2 PC: 0.8280825547067426
Train with iris data set -> mean square error of dataIV with 2 PC: 3.3317221039403275
Train with iris data set -> mean square error of dataV with 2 PC: 0.17556300024433896
Train with iris data set -> mean square error of dataI with 3 PC: 0.9083929073982748
Train with iris data set -> mean square error of dataII 